## AAL atlas

In [1]:
import pandas as pd
import numpy as np
import os
import re
import matplotlib.pyplot as plt
import seaborn as sns
from dfc_functions import PickPickle
import pickle
from dfc_functions import FetchROI

In [5]:
parent_dir = os.path.dirname(os.getcwd()) # get parent directory
pheno_dir = os.path.join(parent_dir, 'phenotype_files') # get directory for phenotype data files
phenotype_path = os.path.join(parent_dir,'phenotype_files/pheno_clean.csv') # path to phenotype file
aal_dir = os.path.join(parent_dir, 'rois_aal')

In [3]:
df = pd.read_csv(phenotype_path) # load the phenotype data

In [4]:
df.head()

,SUB_ID,FILE_ID,AGE_AT_SCAN,SEX,DSM_IV_TR,DX_GROUP,FMRI_FILES,CC200,DFC_DATA_STORE
0,50002,Pitt_0050002,16.77,1,1,1,Pitt_0050002_func_preproc.nii.gz,Pitt_0050002_rois_cc200.1D,Pitt_0050002_dfc.pkl
1,50003,Pitt_0050003,24.45,1,1,1,Pitt_0050003_func_preproc.nii.gz,Pitt_0050003_rois_cc200.1D,Pitt_0050003_dfc.pkl
2,50004,Pitt_0050004,19.09,1,1,1,Pitt_0050004_func_preproc.nii.gz,Pitt_0050004_rois_cc200.1D,Pitt_0050004_dfc.pkl
3,50005,Pitt_0050005,13.73,2,1,1,Pitt_0050005_func_preproc.nii.gz,Pitt_0050005_rois_cc200.1D,Pitt_0050005_dfc.pkl
4,50006,Pitt_0050006,13.37,1,1,1,Pitt_0050006_func_preproc.nii.gz,Pitt_0050006_rois_cc200.1D,Pitt_0050006_dfc.pkl


# check for null values

In [31]:
aal_files = [idx + '_rois_aal.1D' for idx in df.FILE_ID]

In [ ]:
fetch =FetchROI(aal_dir)

In [27]:
null_file = {}
for i,file in enumerate(aal_files):
    data = fetch.fetch_roi_avg_ts(file)
    for f in data.columns:
        val_l = len(data[f].unique())
        if val_l == 1:
            null_file[file] = data[f].unique()

In [53]:
len(null_file)

88

In [51]:
null_idx = []
for i,file in enumerate(aal_files):
    data = fetch.fetch_roi_avg_ts(file)
    null = []
    for f in data.columns:
        val_l = len(data[f].unique())
        if val_l == 1:
            null.append(1)
    if len(null)> 0:
        null_idx.append(i)

In [54]:
len(null_idx)

88

In [55]:
df.iloc[null_idx,:]

,SUB_ID,FILE_ID,AGE_AT_SCAN,SEX,DSM_IV_TR,DX_GROUP,FMRI_FILES,CC200,DFC_DATA_STORE
2,50004,Pitt_0050004,19.09,1,1,1,Pitt_0050004_func_preproc.nii.gz,Pitt_0050004_rois_cc200.1D,Pitt_0050004_dfc.pkl
5,50007,Pitt_0050007,17.78,1,1,1,Pitt_0050007_func_preproc.nii.gz,Pitt_0050007_rois_cc200.1D,Pitt_0050007_dfc.pkl
9,50011,Pitt_0050011,16.93,1,1,1,Pitt_0050011_func_preproc.nii.gz,Pitt_0050011_rois_cc200.1D,Pitt_0050011_dfc.pkl
14,50016,Pitt_0050016,21.82,1,1,1,Pitt_0050016_func_preproc.nii.gz,Pitt_0050016_rois_cc200.1D,Pitt_0050016_dfc.pkl
15,50017,Pitt_0050017,22.70,1,1,1,Pitt_0050017_func_preproc.nii.gz,Pitt_0050017_rois_cc200.1D,Pitt_0050017_dfc.pkl
...,...,...,...,...,...,...,...,...,...
1066,51580,SBL_0051580,42.00,1,3,1,SBL_0051580_func_preproc.nii.gz,SBL_0051580_rois_cc200.1D,SBL_0051580_dfc.pkl
1068,51582,SBL_0051582,31.00,1,2,1,SBL_0051582_func_preproc.nii.gz,SBL_0051582_rois_cc200.1D,SBL_0051582_dfc.pkl
1069,51583,SBL_0051583,35.00,1,2,1,SBL_0051583_func_preproc.nii.gz,SBL_0051583_rois_cc200.1D,SBL_0051583_dfc.pkl
1070,51584,SBL_0051584,49.00,1,2,1,SBL_0051584_func_preproc.nii.gz,SBL_0051584_rois_cc200.1D,SBL_0051584_dfc.pkl


88 subjects contain all missing values in atleast one roi time series, these 88 shall be elimnated

In [56]:
aal_nn = df.drop(null_idx)

In [57]:
aal_nn.reset_index(inplace=True,drop=True)

In [58]:
nn_save_path = os.path.join(pheno_dir, 'aal_nn.csv')

In [59]:
aal_nn.to_csv(nn_save_path, index= False)

In [60]:
pd.read_csv(nn_save_path)

,SUB_ID,FILE_ID,AGE_AT_SCAN,SEX,DSM_IV_TR,DX_GROUP,FMRI_FILES,CC200,DFC_DATA_STORE
0,50002,Pitt_0050002,16.77,1,1,1,Pitt_0050002_func_preproc.nii.gz,Pitt_0050002_rois_cc200.1D,Pitt_0050002_dfc.pkl
1,50003,Pitt_0050003,24.45,1,1,1,Pitt_0050003_func_preproc.nii.gz,Pitt_0050003_rois_cc200.1D,Pitt_0050003_dfc.pkl
2,50005,Pitt_0050005,13.73,2,1,1,Pitt_0050005_func_preproc.nii.gz,Pitt_0050005_rois_cc200.1D,Pitt_0050005_dfc.pkl
3,50006,Pitt_0050006,13.37,1,1,1,Pitt_0050006_func_preproc.nii.gz,Pitt_0050006_rois_cc200.1D,Pitt_0050006_dfc.pkl
4,50008,Pitt_0050008,32.45,1,1,1,Pitt_0050008_func_preproc.nii.gz,Pitt_0050008_rois_cc200.1D,Pitt_0050008_dfc.pkl
...,...,...,...,...,...,...,...,...,...
981,51574,SBL_0051574,30.00,1,2,1,SBL_0051574_func_preproc.nii.gz,SBL_0051574_rois_cc200.1D,SBL_0051574_dfc.pkl
982,51576,SBL_0051576,31.00,1,3,1,SBL_0051576_func_preproc.nii.gz,SBL_0051576_rois_cc200.1D,SBL_0051576_dfc.pkl
983,51581,SBL_0051581,64.00,1,3,1,SBL_0051581_func_preproc.nii.gz,SBL_0051581_rois_cc200.1D,SBL_0051581_dfc.pkl
984,51606,MaxMun_a_0051606,29.00,2,2,1,MaxMun_a_0051606_func_preproc.nii.gz,MaxMun_a_0051606_rois_cc200.1D,MaxMun_a_0051606_dfc.pkl
